In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/sonu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os
import random
from pydub import AudioSegment, silence
import nltk

/opt/homebrew/lib/python3.13/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# ---------------------------
# SETTINGS
# ---------------------------
INPUT_FOLDER = "./data/fluent"
OUTPUT_FOLDER = "./data/stuttered"

In [ ]:
# How likely to repeat a word or syllable (0.0 to 1.0)
REPEAT_PROB = 0.9

# How likely to insert random pause (0.0 to 1.0)
PAUSE_PROB = 0.2

# Pause duration in ms
PAUSE_MS = 300

In [5]:

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [6]:
from pydub.silence import split_on_silence

# ---------------------------
# UTILS
# ---------------------------
def process_file(input_path, output_path):
    audio = AudioSegment.from_wav(input_path)
    chunks = split_on_silence(audio, silence_thresh=-40, min_silence_len=200, keep_silence=100)

    result = AudioSegment.silent(duration=0)

    for chunk in chunks:
        if random.random() < REPEAT_PROB:
            repeats = random.randint(1, 2)
            for _ in range(repeats):
                result += chunk
                result += AudioSegment.silent(duration=50)
        else:
            result += chunk

        if random.random() < PAUSE_PROB:
            result += AudioSegment.silent(duration=PAUSE_MS)

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    result.export(output_path, format="wav")

In [7]:
# ---------------------------
# MAIN
# ---------------------------
for root, _, files in os.walk(INPUT_FOLDER):
    for fname in files:
        if fname.endswith(".wav"):
            rel_path = os.path.relpath(root, INPUT_FOLDER)
            output_dir = os.path.join(OUTPUT_FOLDER, rel_path)
            input_file = os.path.join(root, fname)
            output_file = os.path.join(output_dir, f"stuttered_{fname}")

            print(f"Processing: {input_file} -> {output_file}")
            process_file(input_file, output_file)

print("✅ All done!")

✅ All done!
